In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

import os,json,math,time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('D:\My Documents\Github\data-enrichment-wk14-activity-mapping-yelp-api-results\.secret\yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [15]:
# instantiate the YelpAPI variable
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api

In [4]:
# parameters
LOCATION = 'NJ'
TERM = 'Pizza'

In [5]:
# SPecify JSON_FILE
JSON_FILE = 'D:/My Documents/GitHub/Stack-3/burger_results.json'
JSON_FILE

'D:/My Documents/GitHub/Stack-3/burger_results.json'

In [7]:
# Create directory for JSON FILE
folder = os.path.dirname(JSON_FILE)
os.makedirs(folder,exist_ok = True)

# save an empty list 
with open(JSON_FILE, 'w') as f:
    json.dump([],f)

In [8]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
## How many results total?
total_results = results['total']
total_results

629

In [11]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [12]:
# Use math.ceil to round up for the total number of pages of results 
n_pages = math.ceil((results['total'] - n_results)/results_per_page)
n_pages

32

In [13]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [16]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/32 [00:00<?, ?it/s]

In [17]:
# load results
df = pd.read_json(JSON_FILE)
display(df.head(), df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,0E089bWgJvUl396T693rYg,mercato-tomato-pie-newark,Mercato Tomato Pie,https://s3-media1.fl.yelpcdn.com/bphoto/uKjPYn...,False,https://www.yelp.com/biz/mercato-tomato-pie-ne...,371,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.8,"{'latitude': 40.73486, 'longitude': -74.17068}","[pickup, delivery]",$$,"{'address1': '212 Market St', 'address2': '', ...",+19734249000,(973) 424-9000,101.156843
1,Oh5PRqcZxYN8yH4I0_xWJQ,queen-pizza-ii-newark,Queen Pizza II,https://s3-media4.fl.yelpcdn.com/bphoto/OlH5lp...,False,https://www.yelp.com/biz/queen-pizza-ii-newark...,31,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.7362442016602, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '48 Commerce St', 'address2': Non...",+19732421829,(973) 242-1829,235.020374
2,PxOwmYlSn7hNq9lcu1LLvw,burke-s-tavern-newark-2,Burke’s Tavern,https://s3-media1.fl.yelpcdn.com/bphoto/YQHcsT...,False,https://www.yelp.com/biz/burke-s-tavern-newark...,61,"[{'alias': 'newamerican', 'title': 'New Americ...",4.5,"{'latitude': 40.73066260491095, 'longitude': -...",[],NaN,"{'address1': '150 Lafayette St', 'address2': '...",+19734339883,(973) 433-9883,677.392164
3,CxHEpWA7LKB_NiLto0vHwA,arties-maplewood-2,Artie's,https://s3-media1.fl.yelpcdn.com/bphoto/5HhAN3...,False,https://www.yelp.com/biz/arties-maplewood-2?ad...,865,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.4,"{'latitude': 40.71902090502666, 'longitude': -...",[delivery],$$,"{'address1': '15 Newark Way', 'address2': '', ...",+19733785800,(973) 378-5800,7476.700189
4,wFQ_fqPGUxjLNzGXgldlhQ,ellies-bistro-newark,Ellie's Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/7-VMlC...,False,https://www.yelp.com/biz/ellies-bistro-newark?...,35,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.4,"{'latitude': 40.7322839, 'longitude': -74.1728...","[pickup, delivery]",NaN,"{'address1': '20 Green St', 'address2': None, ...",+19737322385,(973) 732-2385,344.733694


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
624,6AhB6x_tiHkGAd3Ec-qg0A,trend-coffee-and-tea-house-montclair,Trend Coffee & Tea House,https://s3-media4.fl.yelpcdn.com/bphoto/Y5AY4Q...,False,https://www.yelp.com/biz/trend-coffee-and-tea-...,269,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.4,"{'latitude': 40.81317, 'longitude': -74.2162}","[delivery, pickup]",$,"{'address1': '411 Bloomfield Ave', 'address2':...",+19737441333,(973) 744-1333,9433.038200
625,GKg7RPIYxuuannbB2t9lMw,momoya-bloomfield,Momoya,https://s3-media2.fl.yelpcdn.com/bphoto/-uvjDr...,False,https://www.yelp.com/biz/momoya-bloomfield?adj...,278,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.8,"{'latitude': 40.8067425, 'longitude': -74.1928...","[delivery, pickup]",$$,"{'address1': '390 Broad St', 'address2': '', '...",+19737480077,(973) 748-0077,8153.372066
626,LjXiUafOPQnbaa7iccadHw,applebees-grill-bar-union-2,Applebee's Grill + Bar,https://s3-media2.fl.yelpcdn.com/bphoto/x4B9RP...,False,https://www.yelp.com/biz/applebees-grill-bar-u...,140,"[{'alias': 'tradamerican', 'title': 'American'...",2.0,"{'latitude': 40.694038, 'longitude': -74.25798}","[delivery, pickup]",$$,"{'address1': '1721 Morris Ave', 'address2': No...",+19089641070,(908) 964-1070,8590.608884
627,BEr7Io3gaVa4i6Z1er_S7g,happy-wok-jersey-city,Happy Wok,https://s3-media2.fl.yelpcdn.com/bphoto/8JnBbh...,False,https://www.yelp.com/biz/happy-wok-jersey-city...,21,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.1,"{'latitude': 40.7149162, 'longitude': -74.0754...","[delivery, pickup]",NaN,"{'address1': '497 Martin Luther King Dr', 'add...",+12012001090,(201) 200-1090,8424.912991
628,ZEDFx5PDKZtw4yN8MuYnQg,subway-montclair-11,Subway,https://s3-media2.fl.yelpcdn.com/bphoto/lUBwo1...,False,https://www.yelp.com/biz/subway-montclair-11?a...,16,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.8,"{'latitude': 40.8130181072279, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '49 Claremont Ave', 'address2': '...",+19737444444,(973) 744-4444,9147.615670
